In [38]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
from astropy.table import Table, Column, vstack
import json
import shutil

from desitarget.targetmask import desi_mask, obsconditions
from collections import Counter
import subprocess

%matplotlib inline

In [44]:
full_targets = Table.read("targets/subset_dr8_mtl_dark_gray_NGC.fits")
zcat = Table.read("strategy_A/zcat/dark2_dark3_zcat.fits")
full_mtl = Table.read("strategy_A/targets/dark2_dark3_subset_dr8_mtl_dark_gray_northern_cap.fits")

ii = np.in1d(full_mtl['TARGETID'], zcat['TARGETID'])
in_mtl = full_mtl[ii]

ii = np.in1d(full_targets['TARGETID'], zcat['TARGETID'])
in_targets = full_targets[ii]

print(len(zcat), len(in_mtl), len(in_targets))
in_mtl.sort(keys='TARGETID')
zcat.sort(keys='TARGETID')
in_targets.sort(keys='TARGETID')


864678 864678 864678


In [45]:
assert np.all(in_mtl['TARGETID']==zcat['TARGETID'])
assert np.all(in_mtl['TARGETID']==in_targets['TARGETID'])

In [53]:
zcat_large = zcat[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_mtl_large = in_mtl[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
in_targets_large = in_targets[(zcat['NUMOBS']>4) & (zcat['SPECTYPE']!='STAR')]
print(len(zcat_large), 100*len(zcat_large)/len(zcat))

8021 0.9276285507437451


In [47]:
names = {}
for i in range(len(in_mtl_large)):
    a = desi_mask.names(in_mtl_large['DESI_TARGET'][i])
    n = ''
    for aa in a:
        n = n +' '+ aa
    if n not in names.keys():
        names[n] = 1
    else:
        names[n] += 1

In [48]:
names

{' ELG QSO ELG_SOUTH QSO_SOUTH': 4439,
 ' QSO QSO_SOUTH BGS_ANY MWS_ANY': 517,
 ' QSO QSO_SOUTH': 2603,
 ' QSO QSO_SOUTH MWS_ANY': 53,
 ' QSO QSO_SOUTH BGS_ANY': 351,
 ' ELG QSO ELG_SOUTH QSO_SOUTH BGS_ANY': 41,
 ' LRG QSO LRG_1PASS LRG_SOUTH QSO_SOUTH LRG_1PASS_SOUTH': 2,
 ' QSO QSO_SOUTH STD_FAINT BGS_ANY MWS_ANY': 6,
 ' QSO QSO_SOUTH STD_WD BGS_ANY MWS_ANY': 7,
 ' QSO QSO_SOUTH STD_WD MWS_ANY': 1,
 ' QSO QSO_SOUTH STD_FAINT MWS_ANY': 1}

In [49]:
zcat_large[:10]

TARGETID,BRICKNAME,SPECTYPE,Z,ZERR,ZWARN,NUMOBS
int64,str8,str10,float32,float32,int32,int32
35185790652780503,,QSO,3.0,0.0,0,5
35185790665363616,,QSO,3.0,0.0,0,5
35185790707306633,,QSO,3.0,0.0,0,5
35185790732470865,,QSO,3.0,0.0,0,5
35185790745054525,,QSO,3.0,0.0,0,5
35185790816357934,,QSO,3.0,0.0,0,5
35185790816359084,,QSO,3.0,0.0,0,5
35185790820552713,,QSO,3.0,0.0,0,5
35185790917022354,,QSO,3.0,0.0,0,5


In [50]:
in_mtl_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
161.233002888,1.33364370003,35185790652780503,393222,0,0,3400,0.602985043769,4,27053,1,3500,3
161.865398347,1.34992244019,35185790665363616,393222,0,0,3400,0.0800550574739,4,27064,1,3500,3
164.414569492,1.33908312179,35185790707306633,3458764513820803076,262148,1280,3400,0.304924297211,4,27060,1,3500,7
165.932626383,1.30130967706,35185790732470865,393222,0,0,3400,0.501065628796,4,27039,1,3500,3
166.627521067,1.30882467511,35185790745054525,393222,0,0,3400,0.605197366581,4,27037,1,3500,3
170.975000174,1.29897024369,35185790816357934,393222,0,0,3400,0.279449473743,4,26990,1,3500,3
170.903985456,1.34799845752,35185790816359084,262148,0,0,3400,0.796655278902,4,26990,1,3500,1
171.24441092,1.33499597276,35185790820552713,393222,0,0,3400,0.748109124999,4,26990,1,3500,3
176.883939346,1.36774842585,35185790917022354,393222,0,0,3400,0.564418442139,4,26974,1,3500,3


In [51]:
in_targets_large[:10]

RA,DEC,TARGETID,DESI_TARGET,BGS_TARGET,MWS_TARGET,PRIORITY_INIT,SUBPRIORITY,NUMOBS_INIT,HPXPIXEL,NUMOBS_MORE,PRIORITY,OBSCONDITIONS
float64,float64,int64,int64,int64,int64,int64,float64,int64,int64,int64,int64,int32
161.233002888,1.33364370003,35185790652780503,393222,0,0,3400,0.602985043769,4,27053,4,3400,3
161.865398347,1.34992244019,35185790665363616,393222,0,0,3400,0.0800550574739,4,27064,4,3400,3
164.414569492,1.33908312179,35185790707306633,3458764513820803076,262148,1280,3400,0.304924297211,4,27060,4,3400,7
165.932626383,1.30130967706,35185790732470865,393222,0,0,3400,0.501065628796,4,27039,4,3400,3
166.627521067,1.30882467511,35185790745054525,393222,0,0,3400,0.605197366581,4,27037,4,3400,3
170.975000174,1.29897024369,35185790816357934,393222,0,0,3400,0.279449473743,4,26990,4,3400,3
170.903985456,1.34799845752,35185790816359084,262148,0,0,3400,0.796655278902,4,26990,4,3400,1
171.24441092,1.33499597276,35185790820552713,393222,0,0,3400,0.748109124999,4,26990,4,3400,3
176.883939346,1.36774842585,35185790917022354,393222,0,0,3400,0.564418442139,4,26974,4,3400,3
